# 概要
配列のサブ配列の最小値を得る方法のまとめ

例えば次のような配列があるとして

| 0| 1| 2| 3| 4| 5| 6| 7| 8| 9|10|11|12|
|--|--|--|--|--|--|--|--|--|--|--|--|--|
|2 | 4| 5| 3| 6| 4| 5| 9| 5| 4| 1| 2| 3|

これに対してqueryMin(4,9)を実行したら9を返すような関数を作りたい。

ここから先は配列の長さを$N$、クエリーする範囲を$[b,e)$、クエリー範囲の大きさ$e-b$を$M$とする。

In [10]:
import os
import numpy as np
import math
from pprint import pprint

# 動作テスト

In [11]:
def test(Query):
    arr = [2, 4, 5, 3, 6, 4, 5, 9, 5, 4, 1, 2, 3]
    q = Query(arr)
    for b in range(len(arr)):
        for e in range(b+1,len(arr)):
            v0 = q.queryMin(b,e)
            v1 = minInArray(arr,b,e)
            assert(v0 == v1)
    print("TEST [" +  Query.__name__ + "] Pass" )

# パフォーマンステスト

In [12]:
import time
def perf(Queryable):
    #arr = [2, 4, 5, 3, 6, 4, 5, 9, 5, 4, 1, 2, 3]
    size = 16
    arr = np.random.randint(0,100,size).tolist()
    queryable = Queryable(arr)
    #
    start = time.time()
    for ti in range(128):
        for b in range(size):
            for e in range(b+1,size):
                queryable.queryMin(b, e)
    elapsed = time.time() - start
    print( "PERF [" + Queryable.__name__ + "] Time: " + str(elapsed).format("{:%.4f}") + "s")

# ユーティリティ関数

In [13]:
# 配列の指定範囲の最大値を返す
def maxInArray(arr,b,e):
    assert(b<=e)
    mx = arr[b]
    for i in range(b+1,e):
        mx = max(mx, arr[i])
    return mx

# 配列の指定範囲の最小値を返す
def minInArray(arr,b,e):
    assert(b<=e)
    mn = arr[b]
    for i in range(b+1,e):
        mn = min(mn, arr[i])
    return mn

# 1. 最も愚直な方法
もっとも愚直な方法は毎回全ての要素を巡回する方法です。

構築コストは$O(1)$、検索コストは$O(M)$、メモリフットプリントは$O(1)$となります。

In [14]:
# 愚直に全てを巡回
class NaiveQuery:
    def __init__(self, arr):
        self.arr = arr

    def queryMin(self,b,e):
        return minInArray(self.arr,b,e)
# 
test(NaiveQuery)
perf(NaiveQuery)


TEST [NaiveQuery] Pass
PERF [NaiveQuery] Time: 0.025946617126464844s


# 2. 全ての結果をテーブルに格納する

次に簡単な方法は全ての$[i,j)$の組み合わせに対するテーブルを作る。

先の例であれば下記のようなテーブルを作る。

|  | 0| 1| 2| 3| 4| 5| 6| 7| 8| 9|10|11|12|13|
|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|
| 0| -| 2| 2| 2| 2| 2| 2| 2| 2| 2| 2| 1| 1| 1|
| 1| -| -| 4| 4| 3| 3| 3| 3| 3| 3| 3| 1| 1| 1|
| 2| -| -| -| 5| 3| 3| 3| 3| 3| 3| 3| 1| 1| 1|
| 3| -| -| -| 5| 3| 3| 3| 3| 3| 3| 3| 1| 1| 1|
| 4| -| -| -| -| 3| 3| 3| 3| 3| 3| 3| 1| 1| 1|
| 5| -| -| -| -| -| 6| 4| 4| 4| 4| 4| 1| 1| 1|
| 6| -| -| -| -| -| -| 4| 4| 4| 4| 4| 1| 1| 1|
| 7| -| -| -| -| -| -| -| 5| 5| 5| 4| 1| 1| 1|
| 8| -| -| -| -| -| -| -| -| 9| 5| 4| 1| 1| 1|
| 9| -| -| -| -| -| -| -| -| -| 5| 4| 1| 1| 1|
|10| -| -| -| -| -| -| -| -| -| -| 4| 1| 1| 1|
|11| -| -| -| -| -| -| -| -| -| -| -| 1| 1| 1|
|12| -| -| -| -| -| -| -| -| -| -| -| -| 2| 2|
|13| -| -| -| -| -| -| -| -| -| -| -| -| -| 3|

クエリーはこのテーブルから一つ値を引くだけなので定数時間で完了する。

構築コストは$O(N^3)$(?)、検索コストは$O(1)$、メモリフットプリントは$O(N^2)$。

In [15]:
#
class FullTable:
    def __init__(self, arr):
        tblSize = len(arr)
        self.table = np.empty((tblSize+1,tblSize+1),dtype=int)
        self.table.fill(0)
        for b in range(tblSize):
            for e in range(b+1, tblSize+1):
                self.table[b,e] = minInArray(arr,b,e)
        #print(self.table)
    
    def queryMin(self,b,e):
        return self.table[b,e]

# 
test(FullTable)
perf(FullTable)

TEST [FullTable] Pass
PERF [FullTable] Time: 0.003962039947509766s


# 3. Sparse Table

次のような2次元のテーブルを作る。
- その行番号を$k$とすると、$2^k$隣まで考慮した最小値を格納する。
- 行の幅は$N-2^k$となるので、行数は$log(N)$となる。

先ほどの例であれば次のようなテーブルを作る。

|  | 0| 1| 2| 3| 4| 5| 6| 7| 8| 9|10|11|12|
|--|--|--|--|--|--|--|--|--|--|--|--|--|--|
|L0| 2| 4| 5| 3| 6| 4| 5| 9| 5| 4| 1| 2| 3|
|L1| 2| 4| 3| 3| 4| 4| 5| 5| 4| 1| 1| 2|  |
|L2| 2| 3| 3| 3| 4| 4| 4| 1| 1| 1|  |  |  |
|L3| 2| 3| 3| 1| 1| 1|  |  |  |  |  |  |  |

行が増えるに従い列数が劇的に減るので、二次元配列でありながら全要素数は$O(Nlog(N))$とコンパクトになる。

このテーブルからある範囲の最小値をクエリーをするときには次のような手順を踏む。
- 参照する行番号は$log(e-b)$となる。
- 「その行の$b$番目」と、「$e-2^k$番目」のうち小さい方がその範囲の最小値に対応。
- この操作は範囲がどうであれ、2回のテーブルのルックアップなので定数時間で済む。

構築コストは$O(Nlog(N))$(?)、検索コストは$O(1)$、メモリフットプリントは$O(Nlog(N))$。

In [16]:
#
class SparseTable:
    def __init__(self,arr):
        # テーブルの作成
        self.tbl = []
        self.arr = arr
        tblPrv = arr
        for Li in range(1,4) : 
            serachSize = 2**Li
            #print("SR", serachSize)
            tblTmp = []
            for i in range(len(arr)-serachSize+1):
                tblTmp.append(minInArray(arr,i,i+serachSize))
            self.tbl.append(tblTmp)
        #print(self.tbl)

    def queryMin(self,b,e):
        width = e - b
        if width == 1:
            return self.arr[b]
        tblIndex = int(math.log2(width)) - 1
        tblWidth = 2**(tblIndex+1)
        tt = self.tbl[tblIndex]
        return min(tt[b], tt[e-tblWidth])
# 
test(SparseTable)
perf(SparseTable)

TEST [SparseTable] Pass
PERF [SparseTable] Time: 0.013992071151733398s


# 4. Range Tree

次のような付加情報を持たせた二分枝を作る

- それぞれのノードに範囲情報を持たせる
- その子ノードの範囲を上のノードに伝播させることで、そのノードの下の範囲を得られる。

クエリ―時はクエリーの範囲がノードの範囲を完全に外れる/覆う場合は枝刈りができる。

構築コストは$O(Nlog(N)$、検索コストは$O(log(N))$、メモリフットプリントは$O(Nlog(N))$。

In [17]:
# RangeTree
class Node:
    def __init__(self, b, e, mn, n0, n1):
        self.b  = b
        self.e  = e
        self.mn = mn
        self.n0 = n0
        self.n1 = n1

class RangeTree:
    def __init__(self, arr):
        self.root = self.construct(arr, 0, len(arr), 0)
        #print(self.root)
    
    def print(self):
        self.printSub(self.root)

    def printSub(self, node):
        if node.b + 1 == node.e :
            print( "Leaf:", node.mn)
        else :
            print( "Node:(", node.b, ",", node.e, ")")
            self.printSub(node.n0)
            self.printSub(node.n1)

    def construct(self, arr, b, e, depth):
        if b + 1 == e:
            return Node(b, e, arr[b], None, None)
        else : 
            sep = math.ceil((e-b)/2)+b
            n0 = self.construct(arr, b, sep, depth+1)
            n1 = self.construct(arr, sep, e, depth+1) 
            mn = min(n0.mn, n1.mn)
            return Node(b, e, mn, n0, n1)

    def queryMin(self,b,e):
        return self.queryMinSub(self.root, b, e)
    
    def queryMinSub(self, node, b, e):
        # 一切含まれない場合
        if (node.e <= b) or (e <= node.b):
            return None
        # 完全に含まれる場合
        elif (b <= node.b) and (node.e <= e):
            return node.mn
        # 部分的に重なる場合
        else:
            mn0 = self.queryMinSub(node.n0, b, e)
            mn1 = self.queryMinSub(node.n1, b, e)
            if mn0 == None:
                mn2 = mn1
            elif mn1 == None:
                mn2 = mn0
            else:
                mn2 = min(mn0, mn1)
            return mn2

#
test(RangeTree)
perf(RangeTree)

TEST [RangeTree] Pass
PERF [RangeTree] Time: 0.04491400718688965s


# 参考
1. I. Wald, "Computing Minima and Maxima of Subarrays" in Ray Tracing Gems, 2019, ch. 5